<a href="https://colab.research.google.com/github/justinchan114/landsat_clustering/blob/main/Landuse_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=CZNZJkpc1zSNWKhtCTUd3DPgzgWCilSH-heRLP52dZ8&tc=U7yEGwuR2AOTOGEFiBAp-CF-0zfnR7YMn5rFAfeIy2I&cc=fUc5-XkQfqvoa-F0GZ_SZXD6JYDGdZOJs1xWAmQoqHI

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh5pYbiKOQBBaS8f1pU8-_pOS9JVDdpKIvW-XFTEMDuyq3QH_UwF-U0

Successfully saved authorization token.


In [12]:
#Define coordinate of study, an area near Metro Vancouver, BC, Canada
coords = [[-122.4,49.0],
          [-122.0,49.0],
          [-122.0,49.2],
          [-122.4,49.2],
          [-122.4,49.0]]

van = ee.Geometry.Polygon(coords)

#Get the landsat 8 image set(USGS Landsat 8 Collection 2 Tier 1 and Real-Time data Raw Scenes)
input = ee.ImageCollection("LANDSAT/LC08/C02/T1_RT")\
    .filterBounds(van)\
    .select(["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B10", "B11"])

#Computes a Landsat TOA composite from a collection of raw Landsat scenes
#Making image composite to ensure the image could cover the area of study with least clouds
#The way using .sort("CLOUD_COVER").first() will only return one image and does not cover the whole area of study
landsat8 = ee.Algorithms.Landsat.simpleComposite(
    collection = input.filterDate('2018-05-01', '2018-08-31'),
    asFloat=True)

#Making the training dataset
training = landsat8.sample(region=van, scale=30, numPixels=5000)

#Instantiate the clusterer and train it
clusterer = ee.Clusterer.wekaKMeans(6).train(training)

#Cluster the image set(landsat8) using the trained clusterer
result = landsat8.clip(van).cluster(clusterer)

In [13]:
import folium

In [5]:
# Define a function to add a Earth Engine layer to folium
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles = map_id_dict['tile_fetcher'].url_format,
      attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name = name,
      overlay = True,
      control = True
  ).add_to(self)

In [14]:
# Define the visualizer parameters of classification layer
vis_params = {'min': 0, 'max': 5, 'palette': ['red', 'blue', 'white', 'yellow', 'purple', 'green']}


#Parameters for displaying rgb as base map
rgb_params = {'bands' : ["B4","B3","B2"],
              'gamma': 1,
              'max': 0.19,
              'min': 0.02}

# Create a folium map
m = folium.Map(location=[49.1, -122.2], zoom_start=12)

#Add rgb layer
add_ee_layer(m, landsat8.clip(van), rgb_params, 'rgb')

#Add clssification layer
add_ee_layer(m, result.clip(van), vis_params, 'clusters')

#Add NDVI layer
ndvi = landsat8.normalizedDifference(["B5", "B4"]);
ndvi_params = {'min': 0, 
               'max': 1, 
               'palette': ['red', 'yellow', 'green']}
add_ee_layer(m, ndvi.clip(van), ndvi_params, 'NDVI')


m.add_child(folium.LayerControl())

# Display the map
m


Since it is unsupervised clustering, we have to determine the content of each cluster. By viewing the RGB image and NDVI layer, the colours are classified as follows:-

*   Red: Temporary structure/ reflective roof
*   Blue: Water/River
*   White: Urban Area/Farmland with limited plantation
*   Yellow: Sandy surface/ grass and shrubs
*   Purple: Farmland/ Woodland (with lower NDVI)
*   Green: Farmland/ Woodland (with higher NDVI)


